In [2]:
#removing duplicates


In [60]:
import pandas as pd
df = pd.read_csv('validation_kihwa_literature_convertedtosmiles.csv')

In [31]:
df.shape

(70213, 3)

In [32]:
df_no_duplicates = df.drop_duplicates(keep='first')

In [34]:
df_no_duplicates.to_csv('combine_data_no_duplicates.csv', index=False)

In [35]:
#converting csv to txt file

import csv

with open('combine_data_no_duplicates.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    lines = list(csv_reader)

with open('combined_data.txt', 'w') as txt_file:
    for line in lines:
        txt_file.write(' '.join(line) + '\n')
        
        
'''remove column names in combined_data.txt file manually'''       

In [37]:
DATASET='bacteria_data'
radius=2
ngram=3

In [40]:
from collections import defaultdict
import os
import pickle
import sys

import numpy as np

from rdkit import Chem


def create_atoms(mol):
    """Create a list of atom (e.g., hydrogen and oxygen) IDs
    considering the aromaticity."""
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)


def create_ijbonddict(mol):
    """Create a dictionary, which each key is a node ID
    and each value is the tuples of its neighboring node
    and bond (e.g., single and double) IDs."""
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(atoms, i_jbond_dict, radius):
    """Extract the r-radius subgraphs (i.e., fingerprints)
    from a molecular graph using Weisfeiler-Lehman algorithm."""

    if (len(atoms) == 1) or (radius == 0):
        fingerprints = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges
            (i.e., r-radius subgraphs or fingerprints)."""
            fingerprints = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                fingerprints.append(fingerprint_dict[fingerprint])
            nodes = fingerprints

            """Also update each edge ID considering two nodes
            on its both sides."""
            _i_jedge_dict = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    _i_jedge_dict[i].append((j, edge))
            i_jedge_dict = _i_jedge_dict

    return np.array(fingerprints)


def create_adjacency(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    return np.array(adjacency)


def split_sequence(sequence, ngram):
    sequence = '-' + sequence + '='
    words = [word_dict[sequence[i:i+ngram]]
             for i in range(len(sequence)-ngram+1)]
    return np.array(words)


def dump_dictionary(dictionary, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dict(dictionary), f)


if __name__ == "__main__":

    
    radius, ngram = map(int, [radius, ngram])

    with open('combined_data.txt', 'r') as f:
        data_list = f.read().strip().split('\n')

    """Exclude data contains '.' in the SMILES format."""
    data_list = [d for d in data_list if '.' not in d.strip().split()[0]]
    N = len(data_list)

    atom_dict = defaultdict(lambda: len(atom_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))
    word_dict = defaultdict(lambda: len(word_dict))

    Smiles, compounds, adjacencies, proteins, interactions = '', [], [], [], []

    for no, data in enumerate(data_list):

        print('/'.join(map(str, [no+1, N])))

        smiles, sequence, interaction = data.strip().split()
        Smiles += smiles + '\n'

        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))  # Consider hydrogens.
        atoms = create_atoms(mol)
        i_jbond_dict = create_ijbonddict(mol)

        fingerprints = extract_fingerprints(atoms, i_jbond_dict, radius)
        compounds.append(fingerprints)

        adjacency = create_adjacency(mol)
        adjacencies.append(adjacency)

        words = split_sequence(sequence, ngram)
        proteins.append(words)

        interactions.append(np.array([float(interaction)]))

    dir_input = (DATASET + '/cpiprediction/'+
                 'radius' + str(radius) + '_ngram' + str(ngram) + '/')
    os.makedirs(dir_input, exist_ok=True)

    with open(dir_input + 'Smiles.txt', 'w') as f:
        f.write(Smiles)
    np.save(dir_input + 'compounds', compounds)
    np.save(dir_input + 'adjacencies', adjacencies)
    np.save(dir_input + 'proteins', proteins)
    np.save(dir_input + 'interactions', interactions)
    dump_dictionary(fingerprint_dict, dir_input + 'fingerprint_dict.pickle')
    dump_dictionary(word_dict, dir_input + 'word_dict.pickle')

    print('The preprocess of ' + DATASET + ' dataset has finished!')

1/70131
2/70131
3/70131
4/70131
5/70131
6/70131
7/70131
8/70131
9/70131
10/70131
11/70131
12/70131
13/70131
14/70131
15/70131
16/70131
17/70131
18/70131
19/70131
20/70131
21/70131
22/70131
23/70131
24/70131
25/70131
26/70131
27/70131
28/70131
29/70131
30/70131
31/70131
32/70131
33/70131
34/70131
35/70131
36/70131
37/70131
38/70131
39/70131
40/70131
41/70131
42/70131
43/70131
44/70131
45/70131
46/70131
47/70131
48/70131
49/70131
50/70131
51/70131
52/70131
53/70131
54/70131
55/70131
56/70131
57/70131
58/70131
59/70131
60/70131
61/70131
62/70131
63/70131
64/70131
65/70131
66/70131
67/70131
68/70131
69/70131
70/70131
71/70131
72/70131
73/70131
74/70131
75/70131
76/70131
77/70131
78/70131
79/70131
80/70131
81/70131
82/70131
83/70131
84/70131
85/70131
86/70131
87/70131
88/70131
89/70131
90/70131
91/70131
92/70131
93/70131
94/70131
95/70131
96/70131
97/70131
98/70131
99/70131
100/70131
101/70131
102/70131
103/70131
104/70131
105/70131
106/70131
107/70131
108/70131
109/70131
110/70131
111/7013

/home/yaganapu/miniconda3/envs/pl_hpv_latest/lib/python3.9/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


The preprocess of bacteria_data dataset has finished!


In [41]:
len(fingerprint_dict)

1317

In [42]:
len(word_dict)

6422

In [ ]:
# preprocessing of transformercpi


In [ ]:
#word_to_vec model for proteins

In [45]:
# -*- coding: utf-8 -*-
"""
@Time:Created on 2019/4/30 16:10
@author: LiFan Chen
@Filename: word2vec.py
@Software: PyCharm
"""
from gensim.models import Word2Vec
import pandas as pd
import numpy as np


def seq_to_kmers(seq, k=3):
    """ Divide a string into a list of kmers strings.

    Parameters:
        seq (string)
        k (int), default 3
    Returns:
        List containing a list of kmers.
    """
    N = len(seq)
    return [seq[i:i+k] for i in range(N - k + 1)]


class Corpus(object):
    """ An iteratable for training seq2vec models. """

    def __init__(self, dir, ngram):
        self.df = pd.read_csv(dir)
        self.ngram = ngram

    def __iter__(self):
        for sentence in self.df.protein_seq.values:
            yield seq_to_kmers(sentence, self.ngram)


def get_protein_embedding(model,protein):
    """get protein embedding,infer a list of 3-mers to (num_word,100) matrix"""
    vec = np.zeros((len(protein), 100))
    i = 0
    for word in protein:
        vec[i, ] = model.wv[word]
        i += 1
    return vec


if __name__ == "__main__":

    sent_corpus = Corpus("/home/yaganapu/CYP/cyp_update/benchmarks/combined_data_no_duplicates.csv",3)
    model = Word2Vec(vector_size=100, window=5, min_count=1, workers=6)
    model.build_vocab(sent_corpus)
    model.train(sent_corpus,epochs=30,total_examples=model.corpus_count)
    model.save("/home/yaganapu/CYP/cyp_update/benchmarks/bacteria_data/transformercpi/word2vec_30_bacteria.model")

    """
    model = Word2Vec.load("word2vec_30.model")
    vector = get_protein_embedding(model,seq_to_kmers("MSPLNQSAEGLPQEASNRSLNATETSEAWDPRTLQALKISLAVVLSVITLATVLSNAFVLTTILLTRKLHTPANYLIGSLATTDLLVSILVMPISIAYTITHTWNFGQILCDIWLSSDITCCTASILHLCVIALDRYWAITDALEYSKRRTAGHAATMIAIVWAISICISIPPLFWRQAKAQEEMSDCLVNTSQISYTIYSTCGAFYIPSVLLIILYGRIYRAARNRILNPPSLYGKRFTTAHLITGSAGSSLCSLNSSLHEGHSHSAGSPLFFNHVKIKLADSALERKRISAARERKATKILGIILGAFIICWLPFFVVSLVLPICRDSCWIHPALFDFFTWLGYLNSLINPIIYTVFNEEFRQAFQKIVPFRKAS"))
    print(vector.shape)
    """

In [46]:
model = Word2Vec.load("/home/yaganapu/CYP/cyp_update/benchmarks/bacteria_data/transformercpi/word2vec_30_bacteria.model")
vector = get_protein_embedding(model,seq_to_kmers("TTETIQSNANLAPLPPHVPEHLVFDFDMYNPSNLSAGVQEAWAVLQESNVPDLVWTRCNGGHWIATRGQLIREAYEDYRHFSSECPFIPREAGEAYDFIPTSMDPPEQRQFRALANQVVGMPVVDKLENRIQELACSLIESLRPQGQCNFTEDYAEPFPIRIFMLLAGLPEEDIPHLKYLTDQMTRPDGSMTFAEAKEALYDYLIPIIEQRRQKPGTDAISIVANGQVNGRPITSDEAKRMCGLLLVGGLDTVVNFLSFSMEFLAKSPEHRQELIQRPERIPAACEELLRRFSLVADGRILTSDYEFHGVQLKKGDQILLPQMLSGLDERENACPMHVDFSRQKVSHTTFGHGSHLCLGQHLARREIIVTLKEWLTRIPDFSIAPGAQIQHKSGIVSGVQALPLVWDPATTKAV"))

In [47]:
print(vector.shape)

(412, 100)


In [48]:
# -*- coding: utf-8 -*-

import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors

num_atom_feat = 34
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return [x == s for s in allowable_set]


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return [x == s for s in allowable_set]


def atom_features(atom,explicit_H=False,use_chirality=True):
    """Generate atom features including atom symbol(10),degree(7),formal charge,
    radical electrons,hybridization(6),aromatic(1),Chirality(3)
    """
    symbol = ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I', 'other']  # 10-dim
    degree = [0, 1, 2, 3, 4, 5, 6]  # 7-dim
    hybridizationType = [Chem.rdchem.HybridizationType.SP,
                              Chem.rdchem.HybridizationType.SP2,
                              Chem.rdchem.HybridizationType.SP3,
                              Chem.rdchem.HybridizationType.SP3D,
                              Chem.rdchem.HybridizationType.SP3D2,
                              'other']   # 6-dim
    results = one_of_k_encoding_unk(atom.GetSymbol(),symbol) + \
                  one_of_k_encoding(atom.GetDegree(),degree) + \
                  [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + \
                  one_of_k_encoding_unk(atom.GetHybridization(), hybridizationType) + [atom.GetIsAromatic()]  # 10+7+2+6+1=26

    # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
    if not explicit_H:
        results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                      [0, 1, 2, 3, 4])   # 26+5=31
    if use_chirality:
        try:
            results = results + one_of_k_encoding_unk(
                    atom.GetProp('_CIPCode'),
                    ['R', 'S']) + [atom.HasProp('_ChiralityPossible')]
        except:
            results = results + [False, False] + [atom.HasProp('_ChiralityPossible')]  # 31+3 =34
    return results


def adjacent_matrix(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    return np.array(adjacency)+np.eye(adjacency.shape[0])


def mol_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
    except:
        raise RuntimeError("SMILES cannot been parsed!")
    #mol = Chem.AddHs(mol)
    atom_feat = np.zeros((mol.GetNumAtoms(), num_atom_feat))
    for atom in mol.GetAtoms():
        atom_feat[atom.GetIdx(), :] = atom_features(atom)
    adj_matrix = adjacent_matrix(mol)
    return atom_feat, adj_matrix

if __name__ == "__main__":

    #from word2vec import seq_to_kmers, get_protein_embedding
    from gensim.models import Word2Vec
    import os
    """
    atom_feature ,adj = mol_features("C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].[K+].[K+].[Sb+3].[Sb+3]")
    print(atom_feature)
    print(adj)
    """
    DATASET = "bacteria"
    with open("/home/yaganapu/CYP/cyp_update/benchmarks/combined_data.txt","r") as f:
        data_list = f.read().strip().split('\n')
    """Exclude data contains '.' in the SMILES format."""
    data_list = [d for d in data_list if '.' not in d.strip().split()[0]]
    N = len(data_list)
    print(N)

    compounds, adjacencies,proteins,interactions = [], [], [], []
    model = Word2Vec.load("word2vec_30_bacteria.model")
    for no, data in enumerate(data_list):
        print('/'.join(map(str, [no + 1, N])))
        smiles, sequence, interaction = data.strip().split(" ")

        atom_feature, adj = mol_features(smiles)
        compounds.append(atom_feature)
        adjacencies.append(adj)

        interactions.append(np.array([float(interaction)]))

        protein_embedding = get_protein_embedding(model, seq_to_kmers(sequence))
        proteins.append(protein_embedding)
    dir_input = ('/home/yaganapu/CYP/cyp_update/benchmarks/bacteria_data/transformercpi/')
    os.makedirs(dir_input, exist_ok=True)
    np.save(dir_input + 'compounds', compounds)
    np.save(dir_input + 'adjacencies', adjacencies)
    np.save(dir_input + 'proteins', proteins)
    np.save(dir_input + 'interactions', interactions)
    print('The preprocess of ' + DATASET + ' dataset has finished!')

70131
1/70131
2/70131
3/70131
4/70131
5/70131
6/70131
7/70131
8/70131
9/70131
10/70131
11/70131
12/70131
13/70131
14/70131
15/70131
16/70131
17/70131
18/70131
19/70131
20/70131
21/70131
22/70131
23/70131
24/70131
25/70131
26/70131
27/70131
28/70131
29/70131
30/70131
31/70131
32/70131
33/70131
34/70131
35/70131
36/70131
37/70131
38/70131
39/70131
40/70131
41/70131
42/70131
43/70131
44/70131
45/70131
46/70131
47/70131
48/70131
49/70131
50/70131
51/70131
52/70131
53/70131
54/70131
55/70131
56/70131
57/70131
58/70131
59/70131
60/70131
61/70131
62/70131
63/70131
64/70131
65/70131
66/70131
67/70131
68/70131
69/70131
70/70131
71/70131
72/70131
73/70131
74/70131
75/70131
76/70131
77/70131
78/70131
79/70131
80/70131
81/70131
82/70131
83/70131
84/70131
85/70131
86/70131
87/70131
88/70131
89/70131
90/70131
91/70131
92/70131
93/70131
94/70131
95/70131
96/70131
97/70131
98/70131
99/70131
100/70131
101/70131
102/70131
103/70131
104/70131
105/70131
106/70131
107/70131
108/70131
109/70131
110/70131
11

/home/yaganapu/miniconda3/envs/pl_hpv_latest/lib/python3.9/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


The preprocess of bacteria dataset has finished!


In [49]:
len(compounds)

70131

In [50]:
max_compound_length = max(compound.shape[0] for compound in compounds)
#max_compound_length = 70
print(max_compound_length)
max_adjacency_length = max(adjacency.shape[0] for adjacency in adjacencies)
#max_adjacency_length = 70
print(max_adjacency_length)

70
70


In [51]:
num_compounds = len(compounds)
num_compound_chars = compounds[0].shape[1]  # Assuming the second dimension is the number of characters

compound_data = np.zeros((num_compounds, max_compound_length, num_compound_chars))

num_adjacency = len(adjacencies)


adjacency_data = np.zeros((num_adjacency, max_adjacency_length, max_adjacency_length))

In [52]:
for i, compound in enumerate(compounds):
    compound_data[i, :compound.shape[0], :] = compound

for i, adjacency in enumerate(adjacencies):
    adjacency_data[i, :adjacency.shape[0], :adjacency.shape[1]] = adjacency

In [53]:
np.save(dir_input + 'new_padded_compounds', compound_data)
np.save(dir_input + 'new_padded_adjacencies', adjacency_data)

In [54]:
compound_data.shape

(70131, 70, 34)

In [55]:
max_first_dimension = np.max([arr.shape[0] for arr in proteins])

print(f"The maximum value of the first dimension across all arrays is: {max_first_dimension}")

The maximum value of the first dimension across all arrays is: 784


In [56]:
protein_data = np.zeros((num_compounds, max_first_dimension, 100))

In [57]:
for i, compound in enumerate(proteins):
    protein_data[i, :compound.shape[0], :] = compound

In [58]:
protein_data.shape

(70131, 784, 100)

In [59]:
np.save(dir_input + 'new_padded_proteins', protein_data)